# Model Selection and Tuning

This file is concerned with trying different ML algorithms for classification, focusing exclusively on predictive power and starting from the simplest model to the most complicated. We will use the processed data created from the data_preparation.ipynb file, which is a csv named 'train_data_processed.csv'.

The objective of the models is to accurately predict if a client with certain characteristics will default on a loan when asking for it. As we do not know the associated costs with a client not paying back nor the benefits of a client repaying, we will focus on Area Under Curve as the performance metric.

Every model will be trained, tuned and cross-validated using the sci-kit learn library.

In [ ]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

jobs = os.cpu_count()-1 ## A lot of power.

In [ ]:
df = pd.read_csv('train_data_processed.csv', index_col= 0)
df.head()

The data is already scaled for numerical features and encoded (One-hot) for categorical features. We are ready to start working with it.

In [ ]:
df.rename(columns = {col: col.lower() for col in df.columns.values}, inplace = True)
X = df.drop(columns=['target'])
y = df.target.copy()

performance_metrics = {}

### First Model: **Logistic Regression**

For this model we will first fit a logistic regression to all the parameters, and take the mean of the AUC score from a 10 fold cross validation. We will then run a logistic regression regularized by Lasso to try and do feature extraction. We will finally compute the regression without the penalty but using only the parameters extracted from the constrained regression.

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
start = time.time()
logistic_regression_full = SGDClassifier(loss = 'log_loss', l1_ratio= 1, n_jobs= jobs, random_state= 0, early_stopping= True)
results_logistic_regression_full = cross_val_score(logistic_regression_full, X,y, cv = 7, scoring='roc_auc')
end = time.time()
performance_metrics['logistic_regression_full'] = {'error' : np.mean(results_logistic_regression_full),'runtime': end-start}

Regularized Logistic regression

In [ ]:
param_grid = [{'alpha' : [0.0001, 0.001, 0.01, 0.1, 1, 10]}]
logistic_regression_l1 = SGDClassifier(loss = 'log_loss', l1_ratio= 1, n_jobs= jobs, random_state= 0, early_stopping= True)
grid_search = GridSearchCV(logistic_regression_l1, param_grid, cv = 7, scoring = 'roc_auc')
grid_search.fit(X,y)
grid_search.best_params_

As the optimal parameter happens to be the default one, with little to no regularization, we will not remove features using Lasso regression and will accept the original logistic regression as our best model for it.

### Second Model: **Support Vector Machines**

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
svc = SGDClassifier(loss = 'hinge', random_state = 0, max_iter = 10000, early_stopping = True)
param_grid = [{'alpha': [2,2.5,3,3.5]}]
grid_search = GridSearchCV(svc, param_grid, n_jobs = jobs, cv = 7, scoring = 'roc_auc')
grid_search.fit(X,y)
grid_search.best_params_

In [ ]:
start = time.time()
linear_svc = SGDClassifier(alpha = 3, loss = 'hinge', random_state = 0, max_iter = 10000, early_stopping = True)
results_svc = cross_val_score(linear_svc, X,y, cv = 7, n_jobs = jobs, scoring='roc_auc')
end = time.time()
performance_metrics['svc'] = {'error': np.mean(results_svc), 'runtime': end-start}

### Third Model: **Decision Tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
clf = DecisionTreeClassifier(random_state = 0)
param_grid = [{'max_depth': [10,11,12], 'min_samples_leaf': [1200,1400,1600]}]
grid_search = GridSearchCV(clf, param_grid, n_jobs = jobs, cv = 7, scoring = 'roc_auc')
grid_search.fit(X,y)
grid_search.best_params_

In [ ]:
start = time.time()
dec_tree = DecisionTreeClassifier(max_depth = 11, min_samples_leaf = 1400, random_state = 0)
results_decision_tree = cross_val_score(dec_tree, X,y, cv = 7, n_jobs = jobs, scoring='roc_auc')
end = time.time()
performance_metrics['decision_tree'] = {'error' : np.mean(results_decision_tree),'runtime': end-start}

### Fourth Model: **Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
clf = RandomForestClassifier(n_jobs = jobs,random_state= 0)
random_grid = [{'n_estimators': [400,600,800],'max_depth': [15,18,20], 'min_samples_leaf': [1000,1200,1300]}]
grid_search = RandomizedSearchCV(clf, random_grid, cv = 4, scoring = 'roc_auc')
grid_search.fit(X,y)
grid_search.best_params_

In [ ]:
grid_search.best_score_